In [1]:
import torch, shutil, subprocess, textwrap, sys
print("Python exe:", sys.executable)
print("Torch ver:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("torch.version.cuda:", torch.version.cuda)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Python exe: C:\ProgramData\Anaconda3\envs\torchcuda\python.exe
Torch ver: 2.5.1+cu121
CUDA available: True
torch.version.cuda: 12.1
GPU: NVIDIA GeForce GTX 1650


In [2]:
import os, json, random, shutil, sys, time, threading, math, contextlib, subprocess, platform
from pathlib import Path
from typing import Dict, Any, Tuple, List

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from IPython.display import display, HTML

try:
    import pynvml
    pynvml.nvmlInit()
    _NVML_OK = True
except Exception:
    _NVML_OK = False


CSV_FILE = "synthetic_signal_data.csv"
ARTIFACTS_DIR = Path("artifacts_tf")
MODELS_DIR    = Path("Models")
FEATURES = [f"signal_{i}" for i in range(1, 61)]
TIMESTAMP_COL = "Timestamp"

SEQ_LEN = 30
STRIDE  = 1
TEST_SIZE = 0.2
HEARTBEAT_SECS = 30


PARAM_SETS = [
    {"id":"artifact_1",
     "dense":{"lr":1e-3,"batch":512,"epochs":40,"patience":5,"weight_decay":0.0},
     "lstm":{"lr":1e-3,"batch":128,"epochs":40,"patience":5,"hidden":64,"latent":16,"layers":1}},
    {"id":"artifact_2",
     "dense":{"lr":5e-4,"batch":512,"epochs":50,"patience":6,"weight_decay":1e-5},
     "lstm":{"lr":5e-4,"batch":128,"epochs":50,"patience":6,"hidden":96,"latent":24,"layers":1}},
    {"id":"artifact_3",
     "dense":{"lr":1e-3,"batch":1024,"epochs":35,"patience":5,"weight_decay":1e-6},
     "lstm":{"lr":1e-3,"batch":256,"epochs":35,"patience":5,"hidden":80,"latent":20,"layers":1}},
    {"id":"artifact_4",
     "dense":{"lr":2e-3,"batch":512,"epochs":30,"patience":4,"weight_decay":0.0},
     "lstm":{"lr":2e-3,"batch":128,"epochs":30,"patience":4,"hidden":64,"latent":16,"layers":2}},
    {"id":"artifact_5",
     "dense":{"lr":7e-4,"batch":512,"epochs":45,"patience":5,"weight_decay":5e-6},
     "lstm":{"lr":7e-4,"batch":128,"epochs":45,"patience":5,"hidden":72,"latent":18,"layers":1}},
]


SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True  
torch.backends.cudnn.benchmark = False

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
AMP = (DEVICE.type == "cuda")
print("Using:", DEVICE, "| AMP:", AMP)
if DEVICE.type == "cuda":
    try: print("GPU:", torch.cuda.get_device_name(0))
    except: pass

def make_loader(ds,batch,shuffle=True):
    return DataLoader(
        ds,
        batch_size=batch,
        shuffle=shuffle,
        pin_memory=(DEVICE.type=="cuda"),
        num_workers=0
    )


def autocast_ctx():
    if DEVICE.type == "cuda":
        return torch.amp.autocast(device_type="cuda", enabled=AMP)
    return contextlib.nullcontext()


def load_signals_csv(path, features, timestamp_col):
    cols = pd.read_csv(path, nrows=0).columns.tolist()
    usecols = [c for c in features if c in cols] + ([timestamp_col] if timestamp_col in cols else [])
    df = pd.read_csv(path, usecols=usecols)
    for f in features:
        if f not in df: df[f]=0.0
    return df[features].astype("float32").values

class TabularDataset(Dataset):
    def __init__(self, X): self.X = np.asarray(X,dtype=np.float32)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return torch.from_numpy(self.X[idx])

class SequenceDataset(Dataset):
    def __init__(self,X,seq_len=30,stride=1):
        X=np.asarray(X,dtype=np.float32); n=len(X)
        if n<seq_len: self.X=np.empty((0,seq_len,X.shape[1]),dtype=np.float32)
        else:
            num=(n-seq_len)//stride+1
            self.X=np.stack([X[i*stride:i*stride+seq_len] for i in range(num)],axis=0)
    def __len__(self): return len(self.X)
    def __getitem__(self,idx): return torch.from_numpy(self.X[idx])


class DenseAE(nn.Module):
    def __init__(self,in_dim=60):
        super().__init__()
        self.enc=nn.Sequential(nn.Linear(in_dim,40),nn.ReLU(),
                               nn.Linear(40,20),nn.ReLU(),
                               nn.Linear(20,10),nn.ReLU())
        self.dec=nn.Sequential(nn.Linear(10,20),nn.ReLU(),
                               nn.Linear(20,40),nn.ReLU(),
                               nn.Linear(40,in_dim))
    def forward(self,x): return self.dec(self.enc(x))

class LSTMAE(nn.Module):
    def __init__(self,input_dim=60,hidden_dim=64,latent_dim=16,num_layers=1):
        super().__init__()
        self.encoder=nn.LSTM(input_dim,hidden_dim,num_layers=num_layers,batch_first=True)
        self.to_latent=nn.Linear(hidden_dim,latent_dim)
        self.from_latent=nn.Linear(latent_dim,hidden_dim)
        self.decoder=nn.LSTM(input_dim,hidden_dim,num_layers=num_layers,batch_first=True)
        self.out=nn.Linear(hidden_dim,input_dim)
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
    def forward(self,x):
        enc_out,_=self.encoder(x); h_last=enc_out[:,-1,:]
        z=self.to_latent(h_last); base=self.from_latent(z)
        h0=base.unsqueeze(0).repeat(self.num_layers,1,1)
        c0=torch.zeros(self.num_layers, x.size(0), self.hidden_dim, device=x.device, dtype=x.dtype)
        dec_out,_=self.decoder(x,(h0,c0))
        return self.out(dec_out)


def _fmt_eta(seconds):
    if seconds is None or math.isinf(seconds) or seconds < 0: return "—"
    m, s = divmod(int(seconds), 60)
    h, m = divmod(m, 60)
    return f"{h:d}:{m:02d}:{s:02d}" if h else f"{m:02d}:{s:02d}"

class LiveBoard:
    def __init__(self):
        self.html_tpl = """
        <div style="font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, 'Liberation Mono', monospace; max-width: 1000px;">
          <div style="margin-bottom:6px;"><b>Training Dashboard</b> <span style="opacity:.7">| live</span></div>
          <div style="margin:6px 0;">
            <div>Overall <span style="opacity:.7">({overall_desc})</span></div>
            <div style="height:14px;background:#eee;border-radius:7px;overflow:hidden;">
              <div style="height:100%;width:{overall_pct}%;background:#4a90e2;"></div>
            </div>
            <div style="display:flex;justify-content:space-between;font-size:12px;opacity:.85;">
              <div>{overall_n}/{overall_total} ep</div>
              <div>ETA {_overall_eta}</div>
            </div>
          </div>
          <div style="margin:6px 0;">
            <div>Current <span style="opacity:.7">({current_desc})</span></div>
            <div style="height:14px;background:#eee;border-radius:7px;overflow:hidden;">
              <div style="height:100%;width:{current_pct}%;background:#7ed321;"></div>
            </div>
            <div style="display:flex;justify-content:space-between;font-size:12px;opacity:.85;">
              <div>epoch {current_n}/{current_total}</div>
              <div>ETA {_current_eta}</div>
            </div>
          </div>
          <div style="margin-top:6px;font-size:12px;opacity:.9;">
            <span>💓 {heartbeat} </span>
            <span style="margin-left:8px;">phase: {phase}</span>
            <span style="margin-left:8px;">ep: {epoch}</span>
            <span style="margin-left:8px;">tr: {last_tr}</span>
            <span style="margin-left:8px;">va: {last_va}</span>
          </div>
          <div style="margin-top:6px;font-size:12px;opacity:.9;">
            <span><b>GPU</b> util: {gpu_util} | mem: {gpu_mem_used}/{gpu_mem_total} MB | name: {gpu_name}</span>
          </div>
        </div>
        """
        self.state = {
            "overall_desc":"—", "overall_n":0, "overall_total":1, "overall_eta":None,
            "current_desc":"—", "current_n":0, "current_total":1, "current_eta":None,
            "heartbeat":"—", "phase":"init", "epoch":0, "last_tr":"—", "last_va":"—",
            "gpu_util":"—", "gpu_mem_used":"—", "gpu_mem_total":"—", "gpu_name":"—"
        }
        self._display = display(HTML(self._render()), display_id=True)

    def _render(self):
        s = self.state
        return self.html_tpl.format(
            overall_desc=s["overall_desc"],
            overall_pct=0 if s["overall_total"]==0 else int(100*s["overall_n"]/max(1,s["overall_total"])),
            overall_n=s["overall_n"], overall_total=s["overall_total"],
            _overall_eta=_fmt_eta(s["overall_eta"]),
            current_desc=s["current_desc"],
            current_pct=0 if s["current_total"]==0 else int(100*s["current_n"]/max(1,s["current_total"])),
            current_n=s["current_n"], current_total=s["current_total"],
            _current_eta=_fmt_eta(s["current_eta"]),
            heartbeat=s["heartbeat"], phase=s["phase"], epoch=s["epoch"],
            last_tr=s["last_tr"], last_va=s["last_va"],
            gpu_util=s["gpu_util"], gpu_mem_used=s["gpu_mem_used"],
            gpu_mem_total=s["gpu_mem_total"], gpu_name=s["gpu_name"]
        )

    def update(self, **kwargs):
        self.state.update(kwargs)
        self._display.update(HTML(self._render()))


class Heartbeat:
    def __init__(self, interval, board: LiveBoard):
        self.interval = max(5, int(interval))
        self._stop=False
        self.board = board
        self._gpu_name = "—"
        if DEVICE.type == "cuda":
            try:
                self._gpu_name = torch.cuda.get_device_name(0)
            except Exception:
                pass

    def _gpu_stats(self):
        if DEVICE.type != "cuda" or not _NVML_OK:
            return ("—", "—", "—")
        try:
            handle = pynvml.nvmlDeviceGetHandleByIndex(0)
            util = pynvml.nvmlDeviceGetUtilizationRates(handle).gpu  # %
            mem = pynvml.nvmlDeviceGetMemoryInfo(handle)
            used = int(mem.used / 1024**2)
            total = int(mem.total / 1024**2)
            return (f"{util}%", str(used), str(total))
        except Exception:
            return ("—", "—", "—")

    def start(self):
        def loop():
            while not self._stop:
                time.sleep(self.interval)
                if self._stop: break
                util, used, total = self._gpu_stats()
                self.board.update(
                    heartbeat=time.strftime("%H:%M:%S"),
                    gpu_util=util, gpu_mem_used=used, gpu_mem_total=total, gpu_name=self._gpu_name
                )
        self.thr = threading.Thread(target=loop, daemon=True); self.thr.start()
    def stop(self): self._stop=True

class ETA:
    def __init__(self, total, alpha=0.2):
        self.total = max(1,int(total))
        self.done = 0
        self.alpha = alpha
        self._last = time.time()
        self.sec_per_unit = None
    def step(self, n=1):
        now = time.time()
        dt = now - self._last; self._last = now
        if dt <= 0: return
        spu = dt / max(1,n)
        if self.sec_per_unit is None: self.sec_per_unit = spu
        else: self.sec_per_unit = self.alpha*spu + (1-self.alpha)*self.sec_per_unit
        self.done += n
    def remaining(self):
        left = max(0, self.total - self.done)
        if self.sec_per_unit is None: return None
        return left * self.sec_per_unit


def _train_epoch_tabular(model, opt, crit, loader):
    model.train(); tot=0; n=0
    for xb in loader:
        xb = xb.to(DEVICE, non_blocking=True)
        opt.zero_grad(set_to_none=True)
        with autocast_ctx():
            loss = crit(model(xb), xb)
        loss.backward(); opt.step()
        bs = xb.size(0); tot += loss.item()*bs; n += bs
    return tot/max(1,n)

def _eval_tabular(model, crit, loader):
    model.eval(); tot=0; n=0
    with torch.no_grad():
        for xb in loader:
            xb = xb.to(DEVICE, non_blocking=True)
            with autocast_ctx():
                loss = crit(model(xb), xb)
            bs = xb.size(0); tot += loss.item()*bs; n += bs
    return tot/max(1,n)

def _train_epoch_seq(model, opt, crit, loader):
    model.train(); tot=0; n=0
    for xb in loader:
        xb = xb.to(DEVICE, non_blocking=True)
        opt.zero_grad(set_to_none=True)
        with autocast_ctx():
            loss = crit(model(xb), xb)
        loss.backward(); opt.step()
        bs = xb.size(0); tot += loss.item()*bs; n += bs
    return tot/max(1,n)

def _eval_seq(model, crit, loader):
    model.eval(); tot=0; n=0
    with torch.no_grad():
        for xb in loader:
            xb = xb.to(DEVICE, non_blocking=True)
            with autocast_ctx():
                loss = crit(model(xb), xb)
            bs = xb.size(0); tot += loss.item()*bs; n += bs
    return tot/max(1,n)


def train_dense(params,Xtr,Xval,board,overall_eta,hb=None):
    model=DenseAE(Xtr.shape[1]).to(DEVICE)
    opt=torch.optim.AdamW(model.parameters(),lr=params["lr"],weight_decay=params.get("weight_decay",0.0))
    crit=nn.MSELoss()
    tr_loader=make_loader(TabularDataset(Xtr),params["batch"],True)
    va_loader=make_loader(TabularDataset(Xval),params["batch"],False)

    epochs=int(params.get("epochs",10)); patience=int(params.get("patience",5)); no_imp=0
    cur_eta = ETA(total=epochs)
    best=float("inf"); best_state=None

    board.update(current_desc="DenseAE", current_n=0, current_total=epochs, last_tr="—", last_va="—")

    for ep in range(1, epochs+1):
        tr = _train_epoch_tabular(model,opt,crit,tr_loader)
        va = _eval_tabular(model,crit,va_loader)
        if hb: board.update(phase="dense", epoch=ep)

        cur_eta.step(1); overall_eta.step(1)
        board.update(current_n=ep, current_eta=cur_eta.remaining(),
                     overall_n=overall_eta.done, overall_eta=overall_eta.remaining(),
                     last_tr=f"{tr:.4e}", last_va=f"{va:.4e}")

        if va < best - 1e-12:
            best = va; best_state = model.state_dict()
            no_imp = 0
        else:
            no_imp += 1
            if no_imp >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model, float(best)

def train_lstm(params,Xtr,Xval,seq_len,stride,board,overall_eta,hb=None):
    model=LSTMAE(input_dim=Xtr.shape[1], hidden_dim=int(params.get("hidden",64)),
                 latent_dim=int(params.get("latent",16)), num_layers=int(params.get("layers",1))).to(DEVICE)
    opt=torch.optim.AdamW(model.parameters(),lr=params["lr"])
    crit=nn.MSELoss()
    tr_loader=make_loader(SequenceDataset(Xtr,seq_len,stride),params["batch"],True)
    va_loader=make_loader(SequenceDataset(Xval,seq_len,stride),params["batch"],False)

    epochs=int(params.get("epochs",10)); patience=int(params.get("patience",5)); no_imp=0
    cur_eta = ETA(total=epochs)
    best=float("inf"); best_state=None

    board.update(current_desc="LSTMAE", current_n=0, current_total=epochs, last_tr="—", last_va="—")

    for ep in range(1, epochs+1):
        tr = _train_epoch_seq(model,opt,crit,tr_loader)
        va = _eval_seq(model,crit,va_loader)
        if hb: board.update(phase="lstm", epoch=ep)

        cur_eta.step(1); overall_eta.step(1)
        board.update(current_n=ep, current_eta=cur_eta.remaining(),
                     overall_n=overall_eta.done, overall_eta=overall_eta.remaining(),
                     last_tr=f"{tr:.4e}", last_va=f"{va:.4e}")

        if va < best - 1e-12:
            best = va; best_state = model.state_dict()
            no_imp = 0
        else:
            no_imp += 1
            if no_imp >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model, float(best)


ARTIFACTS_DIR.mkdir(exist_ok=True); MODELS_DIR.mkdir(exist_ok=True)

planned_epochs = sum(int(c["dense"].get("epochs",0)) + int(c["lstm"].get("epochs",0)) for c in PARAM_SETS)
board = LiveBoard()
board.update(overall_desc="init", overall_n=0, overall_total=planned_epochs, current_desc="—", current_n=0, current_total=1)

hb = Heartbeat(HEARTBEAT_SECS, board); hb.start()
overall_eta = ETA(total=planned_epochs)

artifacts_index=[]
try:
    X = load_signals_csv(CSV_FILE, FEATURES, TIMESTAMP_COL)
    (ARTIFACTS_DIR / "features.json").write_text(json.dumps({"timestamp_col": TIMESTAMP_COL, "features": FEATURES}, indent=2))
    try:
        schema = pd.read_csv(CSV_FILE, nrows=0).dtypes.apply(lambda t: str(t)).to_dict()
        (ARTIFACTS_DIR / "schema.json").write_text(json.dumps(schema, indent=2))
    except Exception:
        pass

    Xtr, Xval = train_test_split(X, test_size=TEST_SIZE, random_state=SEED, shuffle=True)
    scaler = MinMaxScaler(); Xtr = scaler.fit_transform(Xtr); Xval = scaler.transform(Xval)
    joblib.dump(scaler, ARTIFACTS_DIR/"scaler_prod.pkl")

    if not (ARTIFACTS_DIR/"thresholds.json").exists():
        (ARTIFACTS_DIR/"thresholds.json").write_text(json.dumps({
            "dense": {"method": "percentile", "value": 0.99},
            "lstm":  {"method": "percentile", "value": 0.99, "aggregate": "last"}
        }, indent=2))

    try:
        env_txt = subprocess.run([sys.executable, "-m", "pip", "freeze"], capture_output=True, text=True).stdout
        (ARTIFACTS_DIR / "env.txt").write_text(env_txt)
    except Exception:
        pass
    system = {
        "python": sys.version,
        "torch": torch.__version__,
        "cuda_is_available": torch.cuda.is_available(),
        "cuda_version": torch.version.cuda if hasattr(torch.version, "cuda") else None,
        "cudnn_version": torch.backends.cudnn.version(),
        "device": (torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu"),
        "os": platform.platform(),
    }
    (ARTIFACTS_DIR / "system.json").write_text(json.dumps(system, indent=2))

    for cfg in PARAM_SETS:
        art_id = cfg.get("id","artifact")
        out_dir = MODELS_DIR / art_id
        out_dir.mkdir(parents=True, exist_ok=True)

        board.update(overall_desc=f"{art_id} Dense", phase=f"{art_id}/dense", epoch=0)
        dense_model, dense_val = train_dense(cfg["dense"], Xtr, Xval, board, overall_eta, hb=hb)
        torch.save(dense_model.state_dict(), out_dir/"dense_ae.pt")

        board.update(overall_desc=f"{art_id} LSTM", phase=f"{art_id}/lstm", epoch=0)
        lstm_model, lstm_val = train_lstm(cfg["lstm"], Xtr, Xval, SEQ_LEN, STRIDE, board, overall_eta, hb=hb)
        torch.save(lstm_model.state_dict(), out_dir/"lstm_ae.pt")

        manifest = {
            "id": art_id,
            "input_dim": len(FEATURES),
            "features": FEATURES,
            "seq_len": int(SEQ_LEN),
            "stride": int(STRIDE),
            "aggregate": "last",
            "dense": {
                "val_loss": float(dense_val),
                "batch": int(cfg["dense"]["batch"]),
                "weight_decay": float(cfg["dense"].get("weight_decay", 0.0)),
                "lr": float(cfg["dense"]["lr"]),
                "epochs_planned": int(cfg["dense"]["epochs"]),
                "patience": int(cfg["dense"]["patience"])
            },
            "lstm": {
                "val_loss": float(lstm_val),
                "hidden": int(cfg["lstm"]["hidden"]),
                "latent": int(cfg["lstm"]["latent"]),
                "layers": int(cfg["lstm"]["layers"]),
                "batch": int(cfg["lstm"]["batch"]),
                "lr": float(cfg["lstm"]["lr"]),
                "epochs_planned": int(cfg["lstm"]["epochs"]),
                "patience": int(cfg["lstm"]["patience"])
            },
            "seed": int(SEED),
            "test_size": float(TEST_SIZE)
        }
        (out_dir / "manifest.json").write_text(json.dumps(manifest, indent=2))

        trainlog = {
            "dense": {"val_best": float(dense_val)},
            "lstm":  {"val_best": float(lstm_val)},
            "epochs_planned": int(cfg["dense"]["epochs"]) + int(cfg["lstm"]["epochs"]),
            "finished_at": time.strftime("%Y-%m-%d %H:%M:%S")
        }
        (out_dir / "training_report.json").write_text(json.dumps(trainlog, indent=2))

        artifacts_index.append({"id": art_id, "dense_val": float(dense_val), "lstm_val": float(lstm_val)})

    if artifacts_index:
        best = sorted(artifacts_index, key=lambda r: 0.5*r["dense_val"] + 0.5*r["lstm_val"])[0]["id"]
        (ARTIFACTS_DIR/"registry.json").write_text(json.dumps({"production_artifact": best}, indent=2))

finally:
    hb.stop()

print("Training finished. Artifacts saved under:", ARTIFACTS_DIR.resolve())


Using: cuda | AMP: True
GPU: NVIDIA GeForce GTX 1650


Training finished. Artifacts saved under: C:\Users\Ishaan Tiwari\Desktop\LSTM_&_DENSE_AE_MODELS\artifacts_tf
